In [1]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['CUDA_VISIBLE_DEVICES'] = '4'
#os.environ.pop('CUDA_VISIBLE_DEVICES')
#os.unsetenv('CUDA_VISIBLE_DEVICES')

In [29]:
import glob
import random
import numpy as np
import h5py 
import pandas as pd
import itertools
import time
from collections import defaultdict 
import pickle

from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Flatten, Merge, Dense, Dropout, merge, Reshape, Dropout
from keras.layers import LSTM, Lambda, TimeDistributed, Activation, RepeatVector, Permute, Masking
from keras.layers.advanced_activations import ELU

In [30]:
class ModelDealer:
    def __init__(self, **params):
        self.base = 'dataset_mu10_' if not 'base' in params else params['base']
        self.all_phi= [ 9802,  21363,  38956,  53533,  68110,  50894,  70624,  95756, 125664]
        self.radiuses = [38, 84, 154, 212, 270, 405, 562, 762, 1000 ]
        self.n_layers= 4 if not 'n_layers' in params else params['n_layers']
        self.max_phi= self.all_phi[:self.n_layers] 
        self.downcast=500 if not 'downcast' in params else params['downcast']

        self.hits_per_layer = 20
        self.tracks_candidates = 5
        #n_layers = len(max_phi)
        self.masking_char = -1.
        self.model_label = "all_%dL_%dH_%dC"%(self.n_layers,
                                              self.hits_per_layer,
                                              self.tracks_candidates )
        self.model_label += ('_'+params['label']) if'label'in params else '' 
        if self.downcast:
            self.downcast_phi = [min(self.downcast,i) for i in self.max_phi] ## downcast only, no upcasting
            self.model_label += '_DC%s'%self.downcast
        else:
            self.downcast_phi = self.max_phi
                
        print "Ready for model",self.model_label
        self.round_count = 0

    
    def get_event(self, df, ievent , norm_out= False):
        #print ievent
        sub_data = df.loc[df['event_id']==ievent]
        data = np.full((self.n_layers, self.hits_per_layer), self.masking_char)
        target = np.full( (self.tracks_candidates, self.n_layers), self.masking_char)
        truth = defaultdict(list)
        #large_target = []
        #very_large_target = []
        for layer,w in enumerate(self.downcast_phi):
            for track in range(self.tracks_candidates):
                target[track,layer] = w
                #large_target.append( np.full((1,1), w).astype(int) ) ## the default is the layer len, meaning no hit category
                #very_large_target.append( np.zeros((1,w)) )
                
        per_layer = np.zeros( (self.n_layers,), dtype=int)
        first_phis = [None]*self.tracks_candidates        
        for layer,iphi,aclass in zip(sub_data['layer'].values,sub_data['iphi'].values,sub_data['cluster_id'].values ):
            norm_phi = iphi / float(self.all_phi[layer])

            if layer>= self.n_layers: continue ## more layers than the model can handle
                    
            truth[aclass].append( [layer,iphi] )
            
            if per_layer[layer]>= self.hits_per_layer: continue ## more entries per layer than the mode can handle
            ## that the main input/output value
            #
            data[layer, per_layer[layer] ] = norm_phi
            #trackid[layer, per_layer[layer] ] = aclass
            per_layer[layer]+=1
                    
            downcast_iphi = int(norm_phi*self.downcast_phi[layer]) if self.downcast else iphi
                    
            if aclass>=self.tracks_candidates: continue ## more track than the model can handle
            if first_phis[aclass] ==None: first_phis[aclass] = norm_phi
            
            if norm_out:
                target[aclass,layer] = norm_phi ## for a regression type of thing
            else:
                target[aclass,layer] = downcast_iphi ## for sparse categorical cross entropy style
                #lin_index = layer*self.tracks_candidates + aclass
                #large_target[lin_index][0,0] = downcast_iphi

        ## do some ordering
        for layer,w in enumerate(self.downcast_phi):
            ## order in increasing per layer
            with_data = np.where( data[layer,:] != self.masking_char)
            in_sort = np.argsort( data[layer,:][with_data] )
            data[layer,:len(in_sort)] = data[layer, in_sort ]
        ## order track tragets by phi of first hit
        in_sort = np.argsort( first_phis )
        target = target[in_sort,:]
            
        data = np.expand_dims(data, 0)
        if not norm_out:
            target = target.astype(int)
                            
        vtruth = [] ## a list of lists
        for i,thits in truth.items(): vtruth.append( thits )
        flat_target = list([ target[h,layer].reshape(1,1) for layer,h in itertools.product( range(self.n_layers), range(self.tracks_candidates)) ])
        return data, flat_target, vtruth
        #return data, large_target, vtruth

    def make_dataset(self,df,limit=None):
        n_events = len(set(df['event_id'].values)) if not limit else limit
        print n_events,"to be collected"
        
        data =None
        target = None
        truth = []
        
        start = time.mktime(time.gmtime())
        for count,ievent in enumerate(list(set(df['event_id'].values))[:n_events]):
            d,t,tt = self.get_event(df, ievent)
            
            truth.append( tt ) # list of list of list [event][track][hits]:[layer,iphi]
            if data==None or target == None:
                data = d
                target = t
            else:
                data= np.concatenate((data,d))
                if type( t) == list:
                    for lin_index in range(len( target )):
                        target[lin_index] = np.concatenate((target[lin_index],t[lin_index]))
                else:
                    target = np.concatenate((target,t))
            if (count+1)%1000 == 0:
                now = time.mktime(time.gmtime())
                print count,"so far",now-start,"[s]"
                start = now
        return data,target, truth

    def save_dataset(self,data,target,truth, fname):
        print "Creating dataset in ",fname
        f = h5py.File(fname,'w')
        f['data'] = data
        f['target'] = target
        f.close()
        p = open(fname+'.pkl','w')
        pickle.dump( truth, p )
        p.close()
        
    def get_frame(self,f):
        df = pd.read_csv(f)
        df['event_id'] = df['event_id'].astype(int)
        df['layer'] = df['layer'].astype(int)
        df['iphi'] = df['iphi'].astype(int)
        df['cluster_id'] = df['cluster_id'].astype(int)
        return df

    def convert_file(self,f,fh5,force=False):
        if os.path.isfile(fh5) and not force:
            print fh5,"exists already"
            return
        print "converting",f
        
        df =self.get_frame(f)
        data, target, truth = self.make_dataset(df)
        self.save_dataset( data, target, truth, fh5 )
        
        
    def convert_files(self,specific=None,
                      force=False):
        pattern = '%s%s.csv'%(self.base , '*' if not specific else specific)
        extra = self.model_label
        converted = []
        for f in glob.glob(pattern):
            fh5 = '%s%s.h5'%(f,extra)
            self.convert_file(f,fh5,force=force)
            converted.append( fh5 )
        return converted

    def list_dataset(self):
        pattern = '%s*%s.h5'%(self.base,self.model_label)
        return sorted(glob.glob(pattern))
    
    def read_dataset(self,
                     specific=None,
                     andtruth=False):
        pattern = '%s*%s.h5'%(self.base,self.model_label) if not specific else specific
        
        print pattern
        data = None
        target = None
        truth=[]
        for fname in glob.glob(pattern):
            print fname
            f = h5py.File(fname)
            d = f['data'].value
            t = f['target'].value
            print d[0][0][:3]
            print t[0][0][:3]
            #print data.shape if data!=None else "NA",d.shape
            #print target.shape if target!=None else "NA",t.shape
            if data==None or target == None:
                data = d
                target = t
            else:
                data= np.concatenate((data,d))
                if type( t) == list:
                    for lin_index in range(len( target )):
                        target[lin_index] = np.concatenate((target[lin_index],t[lin_index]))
                else:
                    #try:
                    #    target = np.concatenate((target,t))
                    #except:
                    target = np.concatenate((target, t), axis=1)
            f.close()
            if andtruth:
                try:
                    p=open(fname+'.pkl')
                    truth.extend( pickle.load(p) )
                    p.close()
                except:
                    pass
            target = [target[i,...] for i in range(target.shape[0])]
        return data,target, truth

    def make_model(self):
        self.round_count = 0
        inputs = []
        lstms = []
        main = Input( (self.n_layers,self.hits_per_layer) , name='phi_layers')
        self.hidden_size_1 = 15
        self.hidden_size_2 = 17

        for l in range(self.n_layers):
            ## get a vector for each layer
            i = Lambda(lambda x: x[:,l,:], name = 'split_%d'%l, output_shape = (self.hits_per_layer,))(main)
            ## get the vector into a sequence of single float
            i = Reshape((self.hits_per_layer,1), name='sequence_layer_%d'%l)(i)
            ## mask all zeros to the LSTM
            m = Masking(mask_value=self.masking_char, name='masking_layer_%d'%l)(i)
            ## each sequence of phis by layer goes into its own lstm
            lstm = LSTM(self.hidden_size_1,
                        name='encoder_lstm_%d'%l) ( m ) ## should this lstm be shared between layers ?
            lstm = Reshape( (1,self.hidden_size_1), name='lstm_sequence_%d'%l) (lstm)
            inputs.append( i )
            lstms.append( lstm )

        ##stack the output of all lstms of each layer
        #m = merge(lstms, mode='concat', concat_axis=1)
        m = merge(lstms, mode='concat', concat_axis=1, name='core_merge')
    
        m = Flatten()(m)
        m = RepeatVector( self.tracks_candidates )(m)
        ## reshape it as a sequence of vector of hidden_size
        m = Reshape((self.tracks_candidates*self.n_layers,self.hidden_size_1 ))(m)
        ## run it through the LSTM, returning the full sequence
        m = LSTM(self.hidden_size_2, name='core_lstm', return_sequences=True)(m)
        ## picking up the lstm output time n_layers input vector were used
        m = Lambda(lambda x: x[:,::self.n_layers,:], name = 'every_other_%d'%self.n_layers, output_shape = (self.tracks_candidates,self.hidden_size_2))(m)
        
        outputs=[]
        picks = [Lambda(lambda x: x[:,cand,:], name = 'pick_seq_%d'%(cand), output_shape=(self.hidden_size_2,)) for cand in range(self.tracks_candidates)]
        for l,w in enumerate(self.downcast_phi):
            decoder = Dense(w+1, activation='linear', name='decoder_%d'%l)## the +1 is for the "no hit" category
            sub_outputs = []
            for cand in range(self.tracks_candidates):
                #pick= picks[cand](m)
                outputs.append( Activation('softmax')(decoder(picks[cand](m))))
            
        
        self.model = Model( input=main ,output= outputs)
        self.model.summary()

        self.model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics = ['accuracy'])
        
        return self.model

    def expected_weights(self):
        return 'model_%s_%d_w.h5'%(self.model_label, self.round_count)
    
    def _roll_up(self, data=None, target=None, andTrain=False):
        ##andTrain could be infered from data and target.
        ## will look for the existing weights and train for the next one
        last_weights =None
        base_count = self.round_count
        all_weights = []
        for t in itertools.count():
            self.round_count = base_count+t
            out_weights = self.expected_weights()
            if os.path.isfile( out_weights ):
                last_weights = out_weights
                all_weights.append( out_weights )
                print "\t",out_weights,"found. skipping step"
                continue
            if last_weights:
                print last_weights,"found. Using it"
                self.model.load_weights( last_weights )
            if not andTrain: break
            
            print "Starting training round",self.round_count,time.asctime()
            now = time.mktime(time.gmtime())
            self.model.fit( data, target, batch_size=1000, nb_epoch=100, verbose=0)
            then = time.mktime(time.gmtime())
            print "Trained in",then-now,"[s]"
            self.model.save_weights( out_weights )
            all_weights.append( out_weights )            
            break
        return all_weights
    
    def best_model(self):
        ## load the best existing weights
        return self._roll_up()
        
    def pick_and_train(self, data, target):
        return self._roll_up( data, target, andTrain=True)[-1]
        
    def loop_train(self, data, target,
                   max_round=10,
                   force=False):
        last_weights =None

        base_count = self.round_count
        for t in range(max_round):
            out_weights = self.pick_and_train(data, target)

            ev = self.model.evaluate( data, target, batch_size=1000)
            print "round",t,"/",max_round, ev
            continue

    def qsub_train(self, max_round=10):
        ##drive the training with successive submission to qsub
        return

    def visualize( self,
                   index , data, target, prediction, truths,
                   showCand=True,
                   showTarget=True,
                   showHits=True,
                   showTruth=True,
                   extraTitle=''):
        import matplotlib.pyplot as plt
        import matplotlib.cm as cm
        
        truth = truths[index]
        ## hits is a 9 long list of a hits_per_layer-sequence of iphi values [0,1] <= [0,layer_size]
        hits = data[index]
        ## tracks is a (track_candidates*9)-long list of category : iphi==position, layer_size == no hit
        tracks_l = [t[index] for t in target]
        tracks = np.zeros((self.tracks_candidates, self.n_layers ))
        for layer in range(self.n_layers):
            for itrack in range(self.tracks_candidates):
                tracks[itrack,layer] = tracks_l[layer*self.tracks_candidates + itrack]
                
        ## candidates_prob is a (track_candidates*9)-long list of layer_size-category probabilities
        candidates_prob = [p[index] for p in prediction]
        ## candidates_cat : cast probability into the argmax to have the category
        candidates_cat = [np.argmax(prob) for prob in candidates_prob]
        
        ## candidates can be compared with tracks directly
        candidates = np.zeros((self.tracks_candidates, self.n_layers ))
        plt.figure( figsize=(10,10))
        plt.subplot(aspect='equal')
        for layer in range(self.n_layers):
            plt.gcf().gca().add_artist( plt.Circle((0, 0), self.radiuses[layer],color='g',  fill=False ) )
            for itrack in range(self.tracks_candidates):
                candidates[itrack,layer] = candidates_cat[layer*self.tracks_candidates + itrack]
                        
        ## now you can compare
        truths=[]
        preds= []
        potentials = []
        def make_range( l ):
            if l:
                make_range.xr = max(make_range.xr,max([abs(i[0])*make_range.factor for i in l]))
                make_range.yr = max(make_range.yr,max([abs(i[1])*make_range.factor for i in l]))
        make_range.xr = make_range.yr = 0
        make_range.factor = 1.2
        
        for itrack,t in enumerate(truth):
            truth_t = []
            for layer,iphi in t:
                if layer>=self.n_layers:continue
                nphi = iphi / float(self.max_phi[layer])
                x = self.radiuses[layer]* np.cos( nphi * np.pi*2.)
                y = self.radiuses[layer]* np.sin( nphi * np.pi*2.)
            truth_t.append( (x,y))
            make_range(truth_t)
            if showTruth:
                plt.plot( [i[0] for i in truth_t], [i[1] for i in truth_t],
                          label='truth %d'%itrack,
                          marker='s',
                          fillstyle='none',
                          color='black'
                )
        all_hits = []
        for layer in range(hits.shape[0]):
            for i in range(hits.shape[1]):
                nphi = hits[layer,i]
                if nphi==self.masking_char: continue
                x = self.radiuses[layer]* np.cos( nphi * np.pi*2.)
                y = self.radiuses[layer]* np.sin( nphi * np.pi*2.)
                all_hits.append( (x,y) )
        make_range(all_hits)
        if showHits:
            plt.plot([i[0] for i in all_hits],
                     [i[1] for i in all_hits],
                     label = 'all hits in data',
                     marker = '*',
                     color='black',
                     linestyle='None'
        )

        for itrack in range(self.tracks_candidates):
            #print "candidate",itrack
            ttarget= []
            pred= []
            potential = []
            for layer in range(self.n_layers):
                ## values are in iphi, downcasted
                dc_iphi_pred = candidates[itrack,layer]
                if dc_iphi_pred == self.downcast_phi[layer]:
                    ## no hit predicted
                    pass
                else:
                    nphi_pred = candidates[itrack,layer] / float(self.downcast_phi[layer])
                    x = self.radiuses[layer]* np.cos( nphi_pred * np.pi*2.)
                    y = self.radiuses[layer]* np.sin( nphi_pred * np.pi*2.)
                    pred.append( (x,y) )
                    
                dc_iphi_true = tracks[itrack,layer]
                if dc_iphi_true == self.downcast_phi[layer]:
                    ## no hit expected
                    pass
                else:
                    nphi_true = tracks[itrack,layer] / float(self.downcast_phi[layer])
                    x = self.radiuses[layer]* np.cos( nphi_true * np.pi*2.)
                    y = self.radiuses[layer]* np.sin( nphi_true * np.pi*2.)
                    ttarget.append( (x,y) )
            c=cm.hsv( (itrack+2)/float(self.tracks_candidates+2))
            make_range( ttarget )
            make_range( pred )
            if showTarget:
                plt.plot( [i[0] for i in ttarget], [i[1] for i in ttarget],
                          label='target %d'%itrack,
                          #marker='d',
                          marker='+',
                          fillstyle='none',
                          linestyle='--',
                          #color='black',
                          c=c,
                )
            if showCand:
                plt.plot( [i[0] for i in pred], [i[1] for i in pred],
                          label='cand %d'%itrack,
                          marker='o',c=c)
        plt.ylim((-make_range.yr,make_range.yr))
        plt.xlim((-make_range.xr,make_range.xr))
        plt.title("Event %d %s"%(index,extraTitle))
        plt.legend(loc=(1,0))
        
        plt.show()
                                                                                
    def test(self, one_file):
        c = self.convert_files(one_file)
        data, target, _ = self.read_dataset(c[0])
        self.make_model()
        self.loop_train( data, target , max_round = 3)
        

In [31]:
M = ModelDealer( downcast = 500, base = '/bigdata/shared/TkRamp/dataset100MeV_mu10_')

Ready for model all_4L_20H_5C_DC500


In [32]:
M.make_model()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
phi_layers (InputLayer)          (None, 4, 20)         0                                            
____________________________________________________________________________________________________
split_0 (Lambda)                 (None, 20)            0           phi_layers[0][0]                 
____________________________________________________________________________________________________
split_1 (Lambda)                 (None, 20)            0           phi_layers[0][0]                 
____________________________________________________________________________________________________
split_2 (Lambda)                 (None, 20)            0           phi_layers[0][0]                 
___________________________________________________________________________________________

In [33]:
M.convert_files()

/bigdata/shared/TkRamp/dataset100MeV_mu10_20.csvall_4L_20H_5C_DC500.h5 exists already
converting /bigdata/shared/TkRamp/dataset100MeV_mu10_11.csv
1000 to be collected


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:103: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


999 so far 2.0 [s]
Creating dataset in  /bigdata/shared/TkRamp/dataset100MeV_mu10_11.csvall_4L_20H_5C_DC500.h5
converting /bigdata/shared/TkRamp/dataset100MeV_mu10_10.csv
1000 to be collected
999 so far 2.0 [s]
Creating dataset in  /bigdata/shared/TkRamp/dataset100MeV_mu10_10.csvall_4L_20H_5C_DC500.h5
converting /bigdata/shared/TkRamp/dataset100MeV_mu10_41.csv
1000 to be collected
999 so far 1.0 [s]
Creating dataset in  /bigdata/shared/TkRamp/dataset100MeV_mu10_41.csvall_4L_20H_5C_DC500.h5
converting /bigdata/shared/TkRamp/dataset100MeV_mu10_9.csv
1000 to be collected
999 so far 1.0 [s]
Creating dataset in  /bigdata/shared/TkRamp/dataset100MeV_mu10_9.csvall_4L_20H_5C_DC500.h5
converting /bigdata/shared/TkRamp/dataset100MeV_mu10_25.csv
999 to be collected
Creating dataset in  /bigdata/shared/TkRamp/dataset100MeV_mu10_25.csvall_4L_20H_5C_DC500.h5
converting /bigdata/shared/TkRamp/dataset100MeV_mu10_32.csv
1000 to be collected
999 so far 1.0 [s]
Creating dataset in  /bigdata/shared/TkRamp

['/bigdata/shared/TkRamp/dataset100MeV_mu10_20.csvall_4L_20H_5C_DC500.h5',
 '/bigdata/shared/TkRamp/dataset100MeV_mu10_11.csvall_4L_20H_5C_DC500.h5',
 '/bigdata/shared/TkRamp/dataset100MeV_mu10_10.csvall_4L_20H_5C_DC500.h5',
 '/bigdata/shared/TkRamp/dataset100MeV_mu10_41.csvall_4L_20H_5C_DC500.h5',
 '/bigdata/shared/TkRamp/dataset100MeV_mu10_9.csvall_4L_20H_5C_DC500.h5',
 '/bigdata/shared/TkRamp/dataset100MeV_mu10_25.csvall_4L_20H_5C_DC500.h5',
 '/bigdata/shared/TkRamp/dataset100MeV_mu10_32.csvall_4L_20H_5C_DC500.h5',
 '/bigdata/shared/TkRamp/dataset100MeV_mu10_27.csvall_4L_20H_5C_DC500.h5',
 '/bigdata/shared/TkRamp/dataset100MeV_mu10_35.csvall_4L_20H_5C_DC500.h5',
 '/bigdata/shared/TkRamp/dataset100MeV_mu10_2.csvall_4L_20H_5C_DC500.h5',
 '/bigdata/shared/TkRamp/dataset100MeV_mu10_8.csvall_4L_20H_5C_DC500.h5',
 '/bigdata/shared/TkRamp/dataset100MeV_mu10_49.csvall_4L_20H_5C_DC500.h5',
 '/bigdata/shared/TkRamp/dataset100MeV_mu10_50.csvall_4L_20H_5C_DC500.h5',
 '/bigdata/shared/TkRamp/dat

In [34]:
data, target, _ = M.read_dataset()

/bigdata/shared/TkRamp/dataset100MeV_mu10_*all_4L_20H_5C_DC500.h5
/bigdata/shared/TkRamp/dataset100MeV_mu10_45.csvall_4L_20H_5C_DC500.h5
[ 0.0392777   0.19251173  0.34737809]
[173]
/bigdata/shared/TkRamp/dataset100MeV_mu10_13.csvall_4L_20H_5C_DC500.h5
[ 0.18281983  0.23056519  0.36380331]
[91]
/bigdata/shared/TkRamp/dataset100MeV_mu10_5.csvall_4L_20H_5C_DC500.h5
[ 0.38767598  0.4056315   0.43389104]
[193]
/bigdata/shared/TkRamp/dataset100MeV_mu10_15.csvall_4L_20H_5C_DC500.h5
[ 0.34748011  0.43501326  0.77810651]
[500]
/bigdata/shared/TkRamp/dataset100MeV_mu10_23.csvall_4L_20H_5C_DC500.h5
[ 0.37767802  0.4178739   0.68118751]
[500]
/bigdata/shared/TkRamp/dataset100MeV_mu10_49.csvall_4L_20H_5C_DC500.h5
[ 0.24892879  0.33227913  0.3847174 ]
[166]
/bigdata/shared/TkRamp/dataset100MeV_mu10_48.csvall_4L_20H_5C_DC500.h5
[ 0.0300959   0.14752091  0.16925117]
[15]
/bigdata/shared/TkRamp/dataset100MeV_mu10_43.csvall_4L_20H_5C_DC500.h5
[ 0.0499898   0.07998368  0.13150377]
[101]
/bigdata/shared/T

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:181: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


/bigdata/shared/TkRamp/dataset100MeV_mu10_21.csvall_4L_20H_5C_DC500.h5
[ 0.06570088  0.34044073  0.37992246]
[32]
/bigdata/shared/TkRamp/dataset100MeV_mu10_29.csvall_4L_20H_5C_DC500.h5
[ 0.0504999   0.38196286  0.66996531]
[500]
/bigdata/shared/TkRamp/dataset100MeV_mu10_9.csvall_4L_20H_5C_DC500.h5
[ 0.0821261   0.08284024  0.12517853]
[41]
/bigdata/shared/TkRamp/dataset100MeV_mu10_47.csvall_4L_20H_5C_DC500.h5
[ 0.05896756  0.21801673  0.26923077]
[109]
/bigdata/shared/TkRamp/dataset100MeV_mu10_17.csvall_4L_20H_5C_DC500.h5
[ 0.01009998  0.05988574  0.28524791]
[5]
/bigdata/shared/TkRamp/dataset100MeV_mu10_12.csvall_4L_20H_5C_DC500.h5
[ 0.01183432  0.15078555  0.18118751]
[75]
/bigdata/shared/TkRamp/dataset100MeV_mu10_36.csvall_4L_20H_5C_DC500.h5
[ 0.34911243  0.53560498  0.58335034]
[291]
/bigdata/shared/TkRamp/dataset100MeV_mu10_50.csvall_4L_20H_5C_DC500.h5
[ 0.05692716  0.13211589  0.29442971]
[28]
/bigdata/shared/TkRamp/dataset100MeV_mu10_22.csvall_4L_20H_5C_DC500.h5
[ 0.0010202   0.

In [ ]:
M.pick_and_train(data, target)

Starting training round 0 Wed May 10 09:43:47 2017


In [7]:
M.loop_train(data, target)

Starting training round 0 Wed May  3 03:44:43 2017
Trained in 4563.0 [s]
222990/222990 [==============================] - 19s     